In [1]:
import pandas as pd
import os
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings = ratings.drop(['timestamp'],axis=1)
ratings.head()

,user_id,movie_id,rating
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [2]:
# 3점 이상만 남깁니다.
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [3]:
# rating 컬럼의 이름을 count로 바꿉니다.
ratings.rename(columns={'rating':'count_'}, inplace=True)

In [4]:
ratings['count_']

0          5
1          3
2          3
3          4
4          5
          ..
1000203    3
1000205    5
1000206    5
1000207    4
1000208    4
Name: count_, Length: 836478, dtype: int64

In [5]:
# 영화 제목을 보기 위해 메타 데이터를 읽어옵니다.
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head()

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


In [6]:
print(ratings['movie_id'].nunique())
print(ratings['user_id'].nunique())

movie_count = ratings.groupby('movie_id')['user_id'].count()
movie_count.sort_values(ascending=False).head(30)
#movie_count
#movies['movie_id'].loc[movie_count]
movies.loc[movie_count]

3628
6039


,movie_id,title,genre
2000,2069,"Trip to Bountiful, The (1985)",Drama
551,555,True Romance (1993),Action|Crime|Romance
339,343,"Baby-Sitters Club, The (1995)",Children's
102,104,Happy Gilmore (1996),Comedy
214,216,Billy Madison (1995),Comedy
...,...,...,...
752,762,Striptease (1996),Comedy|Crime
280,283,New Jersey Drive (1995),Crime|Drama
47,48,Pocahontas (1995),Animation|Children's|Musical|Romance
36,37,Across the Sea of Time (1995),Documentary


In [7]:
ratings = pd.merge(ratings, movies[['title', 'movie_id']], on='movie_id', how='left')
ratings

,user_id,movie_id,count_,title
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975)
1,1,661,3,James and the Giant Peach (1996)
2,1,914,3,My Fair Lady (1964)
3,1,3408,4,Erin Brockovich (2000)
4,1,2355,5,"Bug's Life, A (1998)"
...,...,...,...,...
836473,6040,1090,3,Platoon (1986)
836474,6040,1094,5,"Crying Game, The (1992)"
836475,6040,562,5,Welcome to the Dollhouse (1995)
836476,6040,1096,4,Sophie's Choice (1982)


In [8]:
# ratings에 있는 유니크한 영화 개수
ratings['movie_id'].nunique()

3628

In [9]:
# rating에 있는 유니크한 사용자 수
ratings['user_id'].nunique()

6039

In [10]:
#가장 인기 있는 영화 30개(인기순) 영화 id별 별점(시청횟수)많은 순
movie_count = ratings.groupby('movie_id')['count_'].count()
movie_count.sort_values(ascending=False).head(30)

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: count_, dtype: int64

In [11]:
# 내가 선호하는 영화를 5가지 골라서 rating에 추가
my_favorite = ['In time (2011)', 'Kingsman: The Secret Service (2015)', 'The Big Short (2015)' , 'The Dark Knight Rises (2012)', 'Inception (2010)']

my_list = pd.DataFrame({'user_id': ['juhwan']*5, 'title': my_favorite, 'count_':[5]*5})

if not ratings.isin({'user_id':['meissamatar']})['user_id'].any(): 
    ratings = ratings.append(my_list)                         

ratings.tail(10)       

,user_id,movie_id,count_,title
836473,6040,1090.0,3,Platoon (1986)
836474,6040,1094.0,5,"Crying Game, The (1992)"
836475,6040,562.0,5,Welcome to the Dollhouse (1995)
836476,6040,1096.0,4,Sophie's Choice (1982)
836477,6040,1097.0,4,E.T. the Extra-Terrestrial (1982)
0,juhwan,NaN,5,In time (2011)
1,juhwan,NaN,5,Kingsman: The Secret Service (2015)
2,juhwan,NaN,5,The Big Short (2015)
3,juhwan,NaN,5,The Dark Knight Rises (2012)
4,juhwan,NaN,5,Inception (2010)


In [12]:
# 인덱싱
userid_unique = ratings['user_id'].unique()
#movie_unique = ratings['movie_id'].unique()

movie_unique = ratings['title'].unique()

# user, movie indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(userid_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [14]:
# 인덱싱이 잘 되었는지 확인
print(user_to_idx['juhwan'])    
print(movie_to_idx['Inception (2010)'])

6039
3632


In [15]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['title'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie_id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,count_,title
0,0,0,5,One Flew Over the Cuckoo's Nest (1975)
1,0,1,3,James and the Giant Peach (1996)
2,0,2,3,My Fair Lady (1964)
3,0,3,4,Erin Brockovich (2000)
4,0,4,5,"Bug's Life, A (1998)"
...,...,...,...,...
0,6039,3628,5,In time (2011)
1,6039,3629,5,Kingsman: The Secret Service (2015)
2,6039,3630,5,The Big Short (2015)
3,6039,3631,5,The Dark Knight Rises (2012)


In [16]:
# csr_matrix
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count_'], (ratings.user_id, ratings.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Row format>

In [17]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [18]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=100, regularization=0.01, use_gpu=False, iterations=15, dtype=np.float32)

In [19]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.longlong'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [20]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/15 [00:00<?, ?it/s]

In [22]:
juhwan, inception = user_to_idx['juhwan'], movie_to_idx['Inception (2010)']
juhwan_vector, inception_vector = als_model.user_factors[juhwan], als_model.item_factors[inception]

In [23]:
np.dot(juhwan_vector, inception_vector)

2.3680328e-05

In [24]:
dark_knight_rises = movie_to_idx['The Dark Knight Rises (2012)']
dark_knight_rises_vector = als_model.item_factors[dark_knight_rises]
np.dot(juhwan_vector, dark_knight_rises_vector)

0.00016165449

In [25]:
favorite_movie = 'Inception (2010)'
movie_id = movie_to_idx[favorite_movie]
similar_movie = als_model.similar_items(movie_id, N=15)
similar_movie

[(3632, 1.0),
 (3629, 0.9929317),
 (3628, 0.98420113),
 (3631, 0.97855806),
 (3630, 0.97532016),
 (3575, 0.95703405),
 (3576, 0.9566379),
 (3578, 0.95647466),
 (3574, 0.95537996),
 (3579, 0.95483375),
 (3580, 0.9544789),
 (3583, 0.9543267),
 (3572, 0.9541311),
 (3577, 0.9537789),
 (3573, 0.9536938)]

In [26]:
idx_to_movie = {v:k for k,v in movie_to_idx.items()}
[idx_to_movie[i[0]] for i in similar_movie]

['Inception (2010)',
 'Kingsman: The Secret Service (2015)',
 'In time (2011)',
 'The Dark Knight Rises (2012)',
 'The Big Short (2015)',
 "Brother's Kiss, A (1997)",
 'War at Home, The (1996)',
 'Number Seventeen (1932)',
 'Century (1993)',
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 "Another Man's Poison (1952)",
 'Neon Bible, The (1995)',
 "I Don't Want to Talk About It (De eso no se habla) (1993)",
 'Daens (1992)',
 'Male and Female (1919)']

In [27]:
def get_similar_movie(movie_name: str):
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [idx_to_movie[i[0]] for i in similar_movie]
    return similar_movie

# 킹스맨과 비슷한 영화 추천
- 내가 넣은 영화라 내가 넣은 영화들이 추천 되어있다.

In [28]:
get_similar_movie('Kingsman: The Secret Service (2015)')

['Kingsman: The Secret Service (2015)',
 'Inception (2010)',
 'In time (2011)',
 'The Big Short (2015)',
 'The Dark Knight Rises (2012)',
 'War at Home, The (1996)',
 'Number Seventeen (1932)',
 "Brother's Kiss, A (1997)",
 'Last of the High Kings, The (a.k.a. Summer Fling) (1996)',
 'Century (1993)']

# 나에게 영화 추천하기

In [29]:
user = user_to_idx['juhwan']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(461, 0.0034568114),
 (361, 0.0030934336),
 (126, 0.0028344356),
 (41, 0.0026886626),
 (122, 0.0026363782),
 (143, 0.002607638),
 (232, 0.002456027),
 (138, 0.0024522936),
 (29, 0.0024290401),
 (168, 0.0022588046),
 (476, 0.0022583236),
 (182, 0.0021786373),
 (2095, 0.002161658),
 (2381, 0.0021486478),
 (2106, 0.0021449015),
 (1830, 0.0020981843),
 (2048, 0.0020786398),
 (258, 0.0020589703),
 (2229, 0.0020343536),
 (243, 0.0020252992)]

In [30]:
[idx_to_movie[i[0]] for i in movie_recommended]

['Meet the Parents (2000)',
 'Casablanca (1942)',
 'Shakespeare in Love (1998)',
 'Rain Man (1988)',
 'Waking Ned Devine (1998)',
 'Gone with the Wind (1939)',
 'Beautiful Thing (1996)',
 'True Lies (1994)',
 'Close Shave, A (1995)',
 'Naked Gun: From the Files of Police Squad!, The (1988)',
 'Wrong Trousers, The (1993)',
 'Caddyshack (1980)',
 'Taking of Pelham One Two Three, The (1974)',
 'Open Your Eyes (Abre los ojos) (1997)',
 'Phantom of the Opera, The (1943)',
 'Return of the Fly (1959)',
 'Broken Hearts Club, The (2000)',
 'Wild Reeds (1994)',
 'Thieves (Voleurs, Les) (1996)',
 'Ghostbusters (1984)']

# 정리
- 판다스를 활용한 데이터 정리는 항상 전처리가 어려운것 같다.
- 인덱스를 다루고 여러 데이터들을 통합하면서 NAN값도 처리해야하고 무튼 너무 어렵다.
- 추천 알고리즘을 그냥 사용했지만 모델링을 하거나 함수를 변경하는 것이 아니라서 정리가 잘 되지 않는것 같다.